In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors, ticker
import cartopy.crs as ccrs

import pandas as pd
import numpy as np
import scipy as sp
from astropy.table import Table
import astropy.units as u
import astropy.coordinates as coord
import arviz as az
import seaborn as sns

import kinesis as kn
import gapipes as gp

In [2]:
plt.style.use("smoh")

We consider
- astrometry:
    * Gaia DR2
    * Hipparcos-Gaia
- radial velocity:
    * Gaia DR2
    * HARPS
- membership:
    * Léao 2019 = Lindegren 2000 + outlier rejection
    * Gaia collab.
    * Röser 2019 (tails)
    * Meingast 2019 (tails)

In [3]:
def get_hyades_dr2_full():
    # Gaia Collaboration DR2 selection
    datadir = "../data/gaia_dr2_clusters"
    tmp1 = Table.read(
        f"{datadir}/tablea1a.dat", format="ascii.cds", readme=f"{datadir}/ReadMe"
    ).to_pandas()
    tmp1.columns = tmp1.columns.str.lower()
    tmp1 = tmp1.rename(columns={"source": "source_id"})
    tmp2 = pd.read_csv(f"{datadir}/table1a_gaiadr2_full.csv", index_col=0)
    dr2cl = pd.merge(tmp1[["source_id", "cluster"]], tmp2, on="source_id")

    hy_dr2 = dr2cl.groupby("cluster").get_group("Hyades")
    return hy_dr2


gdr2 = get_hyades_dr2_full()[["source_id"]]
gdr2["in_dr2"] = True
print(f"number of sources in Gaia DR2 selection: {len(gdr2)} rows")

number of sources in Gaia DR2 selection: 515 rows


In [4]:
leao = pd.read_csv("../data/leao_table.csv", sep=" ", skipinitialspace=True)
print(f"{len(leao)} rows")
hip_harps = set(leao["HIP"].values)

query_leao_sourceid = """
-- cross-match HIPPARCOS2 sources by id
SELECT
  t.hip, xm.angular_distance, xm.number_of_neighbours, gaia.source_id
FROM TAP_UPLOAD.t AS t
JOIN gaiadr2.hipparcos2_best_neighbour AS xm
  ON xm.original_ext_source_id = t.hip
JOIN gaiadr2.gaia_source AS gaia
  ON xm.source_id = gaia.source_id
"""
leaox = gp.gaia.query(query_leao_sourceid, upload_table_name="t", upload_resource=leao[["ID", "HIP"]])[
    ["source_id", "hip"]
]
leaox["in_leao"] = True

missing_in_gdr2 = hip_harps - set(leaox["hip"])
print(f"leao sources missing in gdr2: {len(missing_in_gdr2)}")
print(f"missing HIP: {missing_in_gdr2}")

71 rows
leao sources missing in gdr2: 8
missing HIP: {19808, 20577, 15300, 19781, 20741, 20745, 22380, 20605}


In [5]:
hy_tails_m19 = Table.read(
    "/home/soh/data/meingast2019_hyades_tails/hyades.dat",
    format="ascii.cds",
    readme="/home/soh/data/meingast2019_hyades_tails/ReadMe",
).to_pandas()
m19 = hy_tails_m19.rename(columns={"Source": "source_id"})[["source_id"]]
m19["source_id"] = m19["source_id"].astype(int)
m19["in_meingast"] = True
print(f"Meingast N={len(m19):4d}")

Meingast N= 238


In [6]:
hy_tails_r19 = Table.read(
    "/home/soh/data/roeser_hyades_tails/stars.dat",
    format="ascii.cds",
    readme="/home/soh/data/roeser_hyades_tails/ReadMe",
).to_pandas()
r19 = hy_tails_r19.rename(columns={"Source": "source_id"})[["source_id"]]
r19["source_id"] = r19["source_id"].astype(int)
r19["in_roser"] = True
print(f"Roeser   N={len(r19):4d}")

Roeser   N=1316


In [7]:
from functools import reduce

dfs = [gdr2, leaox, m19, r19]
out = reduce(lambda left, right: pd.merge(left, right, how="outer"), dfs)

In [8]:
# finally, query DR2 astrometry with source_id
out_full = gp.gaia.query_sourceid(out[['source_id']], columns='gaiadr2.gaia_source.*')
out_full = out_full.merge(out, on='source_id')

In [9]:
out_full = out_full.merge(
    hy_tails_r19[["Source", "Member", "Comment"]].rename(
        columns={
            "Source": "source_id",
            "Member": "Member_r19",
            "Comment": "Comment_r19",
        }
    ),
    how="left",
    on="source_id",
)
out_full = out_full.merge(
    leao.rename(columns=lambda x: x + "_leao" if x != "HIP" else x),
    how="left",
    left_on="hip",
    right_on="HIP",
).drop(columns='HIP')

In [10]:
assert out_full['source_id'].duplicated().sum() == 0
print(f"{len(out_full)} rows in the combined catalog")

1389 rows in the combined catalog


In [11]:
%store out_full

Stored 'out_full' (DataFrame)


In [12]:
print(out_full[['in_dr2', 'in_leao', 'in_meingast', 'in_roser']].count())

in_dr2          515
in_leao          63
in_meingast     238
in_roser       1316
dtype: int64


In [13]:
out_full.to_csv("../data/hyades_full.csv", index=False)